<a href="https://colab.research.google.com/github/mikiyonekura/Fractional-Encoding/blob/main/Fractional_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
!sudo apt install minisat
!pip install python-sat
!pip install japanize_matplotlib
!sudo apt install minisat

Reading package lists... Done
Building dependency tree       
Reading state information... Done
minisat is already the newest version (1:2.2.1-5build2).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
minisat is already the newest version (1:2.2.1-5build2).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.


# Fractional Encoding

In [ ]:
import collections
import itertools
import math

def print_encoding(enc):
  file = open(enc['name']+".cnf",'w')
  file.write("p cnf %d %d\n" % (enc['nvars'], enc['nclauses']))
  for clause in enc['clauses']:
      file.write(' '.join(list(map(str,clause))) + " 0\n")

def neg_all(l):
    return list(map(lambda x: -x, l))

def atmostk_bin(lit, k):
    return list(itertools.combinations(neg_all(lit), k + 1))

In [ ]:
div=0
cnf=[]
tweak_big_house=[]
fractional_count=0

#minitreeの数を返す関数
def minitree_count(k,tweak_list): 
  minitree_num=0
  check_point=[]

  if(tweak_list==[]):
    k=k
  else:
    k=int(len(tweak_list)/2)

  for i in range(1,int(math.log(k, 2))): 
    minitree_num=minitree_num+2**i
    check_point.append(minitree_num)

  return minitree_num,check_point

#補助変数のリストが葉になるようにminitreeを生成
def add_minitree(k,tweak_list):
  global div
  tweak_add=[]
  tweak_house=[]
  x=0

  if(len(tweak_list)==4):
    for i in atmostk_bin(tweak_list, 2):
      cnf.append(i)

  else:
    minitree_num,check_point=minitree_count(k,tweak_list)
    print("minitree_num,check_point:",minitree_num,check_point)

    for i in range(1,minitree_num+1):

      n=2*i-1+((div-1)*1000) 
      m=2*i+((div-1)*1000)
      l=600+2*i-1+((div-1)*1000)
      o=600+2*i+((div-1)*1000)
      a=4*i+1+((div-1)*1000)
      b=4*i+2+((div-1)*1000)
      c=4*i+3+((div-1)*1000)
      d=4*i+4+((div-1)*1000)

      if(i>check_point[len(check_point)-2]or len(check_point)==1):
        if(div!=1):
          a=tweak_list[0+4*x]
          b=tweak_list[1+4*x]
          c=tweak_list[2+4*x]
          d=tweak_list[3+4*x]
          x=x+1

      tweak_add.append(l)
      tweak_add.append(o)

      if(i in check_point):
        tweak_house.append(tweak_add)
        tweak_add=[]

      if(i==1):
        for i in atmostk_bin([n,m,n+2,m+2],2):
          cnf.append(i)
      
      for i in [a,b,c,d]: #終了論文の式(10)をCNFに変換しています
        cnf.append([-i,-l,m,n,o])
        cnf.append([-i,l,m,n,-o])

      for i in list(itertools.combinations([a,b,c,d], 2)): #式(11)
        cnf.append([-i[0],-i[1],-l,m,n,-o])
        cnf.append([-i[0],-i[1],l,-m,n,o])
        cnf.append([-i[0],-i[1],l,m,-n,o])

      for i in list(itertools.combinations([a,b,c,d], 3)): #式(12)
        cnf.append([-i[0],-i[1],-i[2],-l,-m,n,o])
        cnf.append([-i[0],-i[1],-i[2],-l,m,-n,o])
        cnf.append([-i[0],-i[1],-i[2],l,-m,n,-o])
        cnf.append([-i[0],-i[1],-i[2],l,m,-n,-o])

      cnf.append([-a,-b,-c,-d,-l,-m,n,-o]) #式(13)
      cnf.append([-a,-b,-c,-d,-l,m,-n,-o])
      cnf.append([-a,-b,-c,-d,l,-m,-n,o])

      cnf.append([n,m,l,o]) #式(15,16)
      cnf.append([-n,-m,-l,-o])

  tweak_house.reverse()
  return tweak_house

#mainの関数(再帰関数)
def fractional(k,n,tweak_list):
  global div,tweak_big_house
  div=div+1 #fractional実行のたびに変数の空間を変える
  tmp=div-1
  
  print("***k,n,div,tweak_list",k,n,div,tweak_list,"***")

  tweak_big_house.append(add_minitree(k,tweak_list))
  print("tweak_big_house:",tweak_big_house)

  for i in range(1,int(math.log(k, 2))): #AM-k/n = AM-(k/2)/(n/2)+AM-(k/4)/(n/4)...+AM-2/4
    print("tmp",tmp)
    print("k,n,i,int(n/(2**i)):",k,n,i,int(n/(2**i))) 
    fractional(int(k/(2**i)),int(n/(2**i)),tweak_big_house[tmp][i-1])#ここで修了論文Step4の制約を再帰的に追加しています

  to_1vec=list(itertools.chain.from_iterable(cnf))
  nvars=len(collections.Counter(to_1vec))
  nliterals=len(to_1vec)


  return {'name': ("fractional_k%d_n%d"%(k,n)),
          'nvars': nvars,
          'nliterals':nliterals,
          'nclauses': len(cnf),
          'clauses': cnf}


In [ ]:
div=0
cnf=[]
tweak_list=[]
tweak_big_house=[]
fractional_count=0

k=128
n=256

#rank=1: 2/4 rank=2: 4/8(5-12) rank=3: 8/16(13-28) 
#rank=4: 16/32(29-60) rank=5: 32/64(61-124) rank=6: 64/128(125-252) rank=7: 128/256(253-508)

for i in range(k-1):
  cnf.append([253+i])

enc=fractional(k,n,tweak_list)
print_encoding(enc)


!minisat fractional_k128_n256.cnf ans.cnf

***k,n,div,tweak_list 128 256 1 [] ***
minitree_num,check_point: 126 [2, 6, 14, 30, 62, 126]
tweak_big_house: [[[725, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852], [661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 